<a href="https://colab.research.google.com/github/Priom7/ML/blob/main/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers --quiet
!pip install opendatasets --quiet

import opendatasets as od

od.download('https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection?select=Sarcasm_Headlines_Dataset.json')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: mdsharifalam
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/rmisra/news-headlines-dataset-for-sarcasm-detection


100%|██████████| 3.30M/3.30M [00:00<00:00, 560MB/s]

In [3]:
import torch
import torch.nn as nn
from torch.optim import Adam
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cuda


In [15]:
data_df = pd.read_json('/content/news-headlines-dataset-for-sarcasm-detection/Sarcasm_Headlines_Dataset.json', lines= True)
data_df.head()
print(data_df.shape)
data_df.dropna(inplace=True)
print(data_df.shape)
data_df.drop_duplicates(inplace=True)
print(data_df.shape)
print(data_df.head())
data_df.drop(['article_link'], inplace=True, axis =1)
print(data_df.head())

(26709, 3)
(26709, 3)
(26708, 3)
                                        article_link  ... is_sarcastic
0  https://www.huffingtonpost.com/entry/versace-b...  ...            0
1  https://www.huffingtonpost.com/entry/roseanne-...  ...            0
2  https://local.theonion.com/mom-starting-to-fea...  ...            1
3  https://politics.theonion.com/boehner-just-wan...  ...            1
4  https://www.huffingtonpost.com/entry/jk-rowlin...  ...            0

[5 rows x 3 columns]
                                            headline  is_sarcastic
0  former versace store clerk sues over secret 'b...             0
1  the 'roseanne' revival catches up to our thorn...             0
2  mom starting to fear son's web series closest ...             1
3  boehner just wants wife to listen, not come up...             1
4  j.k. rowling wishes snape happy birthday in th...             0


In [24]:
X = data_df.iloc[:,:-1]
# print(X.head())
y = data_df.iloc[:, -1]
# print(y.head())

X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.3)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

print("Training Size: " , X_train.shape[0], " and " , round(X_train.shape[0]/X.shape[0]*100), "%")
print("Validation Size: " , X_val.shape[0], " and " , round(X_val.shape[0]/X.shape[0]*100), "%")
print("Testing Size: " , X_test.shape[0], " and " , round(X_test.shape[0]/X.shape[0]*100), "%")



Training Size:  18695  and  70 %
Validation Size:  4007  and  15 %
Testing Size:  4006  and  15 %


In [25]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")
bert_model = AutoModel.from_pretrained("google-bert/bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
class dataset(Dataset):
  def __init__(self, X, y):
    self.X = [tokenizer(x,
                        max_length = 100,
                        trancation = True,
                        padding = 'max_length',
                        return_tensors = 'pt'
                        ).to(device)
                        for x in X
              ]

